In [2]:
import os
import docx
import io
import sys
import win32print
from tkinter import *
import tkinter as tk

doc =docx.Document('boceto.docx')


def convert_to_words(num):
 
    # Get number of digits
    # in given number
    l = len(num)
    #c = ""
    # Base cases
    if (l == 0):
        print("vacio")
        return
 
    if (l > 4):
        print("Mas de 4 digitos gasta mucha memoria :(")
        return
 
    # The first string is not used,
    # it is to make array indexing simple
    single_digits = ["cero", "uno", "dos", "tres",
                     "cuatro", "cinco", "seis", "siete",
                     "ocho", "nueve"]
 
    # The first string is not used,
    # it is to make array indexing simple
    two_digits = ["", "diez", "once", "doce",
                  "trece", "catorce", "quince",
                  "dieciseis", "decisiete", "deciocho",
                  "diecinueve"]
 
    # The first two string are not used,
    # they are to make array indexing simple
    tens_multiple = ["", "", "veinti", "treinta y ", "cuarenta y ",
                     "cincuenta y ", "sesenta y ", "setenta y ", "ochenta y ",
                     "noventa y "]
 
    tens_power = ["cientos", "mil"]
    ten_power = ["ciento", "mil"]
 
    # Used for debugging purpose only
#    print(num, ":", end=" ")
 
    # For single digit number
    if (l == 1):
        print(single_digits[ord(num[0]) - 48])
        return
        #c= single_digits[ord(num[0]) - 48]
        #return str(c)
 
    # Iterate while num is not '\0'
    x = 0
    r = 0
    while (x < len(num)):
 
        # Code path for first 2 digits
        if (l >= 3):
            if (ord(num[x]) - 48 != 0):
                if (ord(num[x]) - 48 == 1):
                    print(ten_power[l - 3], end=" ")
                else:
                    print(single_digits[ord(num[x]) - 48],
                      end="")
                    print(tens_power[l - 3], end=" ")
            l -= 1
 
        # Code path for last 2 digits
        else:
 
            # Need to explicitly handle
            # 10-19. Sum of the two digits
            # is used as index of "two_digits"
            # array of strings
            if (ord(num[x]) - 48 == 1):
                sum = (ord(num[x]) - 48 +
                       ord(num[x+1]) - 48)
                print(two_digits[sum])
                return
            
 

             # especificamos los acompanados de 0
            elif (ord(num[x]) - 48 == 2 and
                  ord(num[x + 1]) - 48 == 0):
                print("veinte")
                return
            elif (ord(num[x]) - 48 == 3 and
                  ord(num[x + 1]) - 48 == 0):
                print("treinta")
                return
            elif (ord(num[x]) - 48 == 4 and
                  ord(num[x + 1]) - 48 == 0):
                print("cuarenta")
                return
            elif (ord(num[x]) - 48 == 5 and
                ord(num[x + 1]) - 48 == 0):
                print("cincuenta")
                return
            elif (ord(num[x]) - 48 == 6 and
                  ord(num[x + 1]) - 48 == 0):
                print("sesenta")
                return
            elif (ord(num[x]) - 48 == 7 and
                  ord(num[x + 1]) - 48 == 0):
                print("setenta")
                return
            elif (ord(num[x]) - 48 == 8 and
                ord(num[x + 1]) - 48 == 0):
                print("ochenta")
                return
            elif (ord(num[x]) - 48 == 9 and
               ord(num[x + 1]) - 48 == 0):
                print("noventa")
                
                return 

            # Rest of the two digit
            # numbers i.e., 21 to 99
            else:
                i = ord(num[x]) - 48
                if(i > 0):
#entra cuando es menor a 99
                    
                    print(tens_multiple[i], end="")
                    
                else:
#entra cuando es = o mayor que cien
                    print("", end="")
                x += 1
                if(ord(num[x]) - 48 != 0):
                    print(single_digits[ord(num[x]) - 48])
        x += 1
        

#---------- interfaz grafica------------------
raiz = Tk()


raiz.title("Foliador de notaria")
#raiz.geometry("650x350")
#raiz.resizable(False,False)
raiz.config(bg="blue")
#raiz.attributes('-alpha', 0.7)
#miframe=Frame(width="650" , height="350")
miFrame= Frame()

miFrame.pack()
miFrame.config(bg="#f4f1f1")
miFrame.config(bd=20)
miFrame.config(width="670" , height="300")
miFrame.config(relief="groove")




#labels
Label(miFrame, text="Pagina Inicial",bg="#f4f1f1", font=("Comic Sans MS",12)).place(x=30, y=30)
Label(miFrame, text="Pagina Final",bg="#f4f1f1", font=("Comic Sans MS",12)).place(x=370, y=30)

#imagen
miImagen=PhotoImage(file="logo,notaria.png")
Label(miFrame, image=miImagen ,bg="#f4f1f1").place(x=10, y=65)


#cuadro de texto

cuadroInicial=Entry(miFrame )
cuadroInicial.place(x=140, y=35)



cuadroFinal=Entry(miFrame)
cuadroFinal.place(x=465, y=35)

#boton de envio
def codigoBoton():
    salto=0
    #inputs
    
    pinicial = int(cuadroInicial.get())

    pfinal = int(cuadroFinal.get()) 
    
    Label(miFrame, text="Procesando!", pady=20).place(x=330, y=220)
    if(pinicial <= pfinal):    
    #convertir numero a string para poder usar convert_to_words
        num = str(pinicial)
    #converimos todo el print de la funcion para utilizarlo como variable
        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        convert_to_words(num)
        output = new_stdout.getvalue()
        sys.stdout = old_stdout
        #insertamos numero y output en principio
        if(pinicial != 100):
            doc.paragraphs[0].add_run("\t \t \t \t \t \t \t{} {}".format(pinicial,output))
        else:
            doc.add_paragraph("\t \t \t \t \t \t \t100 cien")
        while salto <= 32:
            doc.add_paragraph('')
            salto= salto +1
        pinicial=pinicial + 1
        salto= 0
        #####################################
#recorre paginas
        while  pinicial <= pfinal:
#convertir numero a string para poder usar convert_to_words
            num = str(pinicial)
#converimos todo el print de la funcion para utilizarlo como variable
            old_stdout = sys.stdout
            new_stdout = io.StringIO()
            sys.stdout = new_stdout   
            convert_to_words(num)
            output = new_stdout.getvalue()
            sys.stdout = old_stdout
#insertamos numero y output en principio
            if(pinicial != 100):
                doc.add_paragraph("\t \t \t \t \t \t \t{} {}".format(pinicial,output))
            else:
                doc.add_paragraph("\t \t \t \t \t \t \t100 cien")
        #salto siguiente pagina
            while salto <= 31:
                doc.add_paragraph('')
                salto= salto +1
            pinicial= pinicial +1
            salto= 0
    else:
        print("pagina inicial mas grande que pagina final ")
    
#--------------consola------------------
   

    
botonEnvio=Button(miFrame,text="Aceptar", command=codigoBoton,).place(x=315, y=200)
botonEnvio=Button(miFrame,text="Salir", command=raiz.destroy,).place(x=370, y=200)
raiz.mainloop()


doc.save('foliador.docx')

